### Подключение к базе и таблицы с юзерами и постами

In [1]:
from dotenv import load_dotenv
from sqlalchemy.engine import URL
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
load_dotenv()

True

In [3]:
connection = os.environ.get("POSG_ENG")

In [4]:
### Посты и топики
### С постами придется повозиться, сгенерируем эмбеддинги постов с помощью моделей transformers
import pandas as pd


posts_info = pd.read_sql(
    """SELECT * FROM public.post_text_df""",
    con=connection
)

posts_info

,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business
...,...,...,...
7018,7315,"OK, I would not normally watch a Farrelly brot...",movie
7019,7316,I give this movie 2 stars purely because of it...,movie
7020,7317,I cant believe this film was allowed to be mad...,movie
7021,7318,The version I saw of this film was the Blockbu...,movie


In [5]:
from transformers import AutoTokenizer
from transformers import BertModel
from transformers import RobertaModel
from transformers import DistilBertModel

# Функция для возврата предобученной модели и токенизатора

def get_model(model_name):
    # Проверяем, есть ли модель в наличии
    assert model_name in ['bert', 'roberta', 'distilbert']

    checkpoint_names = {
        'bert': 'bert-base-cased',
        'roberta': 'roberta-base',
        'distilbert': 'distilbert-base-cased'
    }

    model_classes = {
        'bert': BertModel,
        'roberta': RobertaModel,
        'distilbert': DistilBertModel
    }

    return AutoTokenizer.from_pretrained(checkpoint_names[model_name]), model_classes[model_name].from_pretrained(checkpoint_names[model_name])

In [6]:
tokenizer, model = get_model('distilbert')

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
### Сделаем датасет для постов

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding


class PostDataset(Dataset):
    def __init__(self, texts, tokenizer):
        super().__init__()

        self.texts = tokenizer.batch_encode_plus(
            texts,
            add_special_tokens=True,
            return_token_type_ids=False,
            return_tensors='pt',
            truncation=True,
            padding=True
        )
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        return {'input_ids': self.texts['input_ids'][idx], 'attention_mask': self.texts['attention_mask'][idx]}

    def __len__(self):
        return len(self.texts['input_ids'])
    
    
dataset = PostDataset(posts_info['text'].values.tolist(), tokenizer)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

loader = DataLoader(dataset, batch_size=32, collate_fn=data_collator, pin_memory=True, shuffle=False)

In [8]:
import torch
from tqdm import tqdm


@torch.inference_mode()
def get_embeddings_labels(model, loader):
    model.eval()
    
    total_embeddings = []
    
    for batch in tqdm(loader):
        batch = {key: batch[key].to(device) for key in ['attention_mask', 'input_ids']}

        embeddings = model(**batch)['last_hidden_state'][:, 0, :]

        total_embeddings.append(embeddings.cpu())

    return torch.cat(total_embeddings, dim=0)

In [9]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)
print(torch.cuda.get_device_name())

model = model.to(device)

cuda:0
NVIDIA GeForce RTX 3050 Ti Laptop GPU


In [10]:
embeddings = get_embeddings_labels(model, loader).numpy()

embeddings

100%|████████████████████████████████████████████████████████████████████████████████| 220/220 [01:52<00:00,  1.96it/s]


array([[ 3.63150865e-01,  4.89376076e-02, -2.64081180e-01, ...,
        -1.41593322e-01,  1.59181338e-02,  9.17690195e-05],
       [ 2.36416355e-01, -1.59500718e-01, -3.27798098e-01, ...,
        -2.89936095e-01,  1.19365320e-01, -1.62343075e-03],
       [ 3.75191331e-01, -1.13944076e-01, -2.40547031e-01, ...,
        -3.38919759e-01,  5.86940572e-02, -2.12656837e-02],
       ...,
       [ 3.40382695e-01,  6.64923638e-02, -1.63184494e-01, ...,
        -8.65628794e-02,  2.03403875e-01,  3.20906416e-02],
       [ 4.32091892e-01,  1.10916262e-02, -1.17306069e-01, ...,
         7.54015967e-02,  1.02739766e-01,  1.52742090e-02],
       [ 3.04277748e-01, -7.62156770e-02, -6.77585602e-02, ...,
        -5.43488115e-02,  2.44383752e-01, -1.41487354e-02]], dtype=float32)

In [11]:
### Кластеризуем тексты

from sklearn.decomposition import PCA

centered = embeddings - embeddings.mean()

pca = PCA(n_components=50)
pca_decomp = pca.fit_transform(centered)

In [12]:
from sklearn.cluster import KMeans

n_clusters = 15

kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(pca_decomp)

posts_info['TextCluster'] = kmeans.labels_

dists_columns = [f'DistanceToCluster_{i}' for i in range(n_clusters)]

dists_df = pd.DataFrame(
    data=kmeans.transform(pca_decomp),
    columns=dists_columns
)

dists_df.head()

,DistanceToCluster_0,DistanceToCluster_1,DistanceToCluster_2,DistanceToCluster_3,DistanceToCluster_4,DistanceToCluster_5,DistanceToCluster_6,DistanceToCluster_7,DistanceToCluster_8,DistanceToCluster_9,DistanceToCluster_10,DistanceToCluster_11,DistanceToCluster_12,DistanceToCluster_13,DistanceToCluster_14
0,3.372981,3.449238,3.613279,2.455638,1.475142,3.410075,3.401131,3.605847,3.482929,1.889863,2.990790,3.417781,2.330397,3.443002,2.839993
1,3.327286,3.129955,3.375149,2.374099,2.187458,3.291418,3.440351,3.347250,3.258184,1.672582,2.841858,3.274209,2.251417,2.983633,2.548337
2,3.263707,3.136165,3.334628,2.418176,2.277349,3.308573,3.549620,3.340957,3.403425,1.582005,3.051923,3.366638,3.114794,2.971580,2.866056
3,3.517888,3.770270,3.167191,2.842223,2.716161,3.736576,3.747341,3.791014,4.074710,2.356338,3.256052,3.790800,3.462207,3.721126,3.368291
4,3.031014,2.830061,3.065462,2.014575,2.055249,2.789516,2.850697,3.022428,3.245411,1.733670,2.643581,2.753824,3.005219,2.636937,2.116556


In [13]:
# Из таблицы постов для контрольной модели заберем столбец TotalTfIdf

post_df = pd.read_sql('SELECT * FROM a_maslennikov_post_features_lesson_22', con=connection)
post_df

,post_id,topic,count_words,first_sent_Tsne_1st_component,first_sent_Tsne_2nd_component,stop_Tsne_1st_component,stop_Tsne_2nd_component,tf_idf_stop_max,tf_idf_stop_mean,TotalTfIdf,...,DistanceTo9thCluster,DistanceTo10thCluster,DistanceTo11thCluster,DistanceTo12thCluster,DistanceTo13thCluster,DistanceTo14thCluster,DistanceTo15thCluster,embedding_1,embedding_2,embedding_3
0,1,business,318,-0.038906,-2.144652,42.836586,-37.384290,0.462776,0.000495,8.748129,...,0.438651,0.260549,0.423094,0.455818,0.545878,0.529774,0.498790,-0.782232,-2.211454,-0.206105
1,2,business,441,-0.193684,-0.233893,52.825260,-14.826131,0.452829,0.004566,11.878472,...,0.296305,0.228026,0.189926,0.340904,0.458724,0.428792,0.395859,-0.670262,-1.993636,-0.072089
2,3,business,538,-2.080238,0.541324,49.007130,-25.862959,0.064370,0.000009,12.675530,...,0.320943,0.094624,0.298398,0.338253,0.478707,0.433204,0.396169,-0.641463,-1.735066,-0.448774
3,4,business,170,-1.724925,-0.322325,59.282013,-26.647335,0.104535,0.000015,6.622786,...,0.262113,0.120913,0.220605,0.301639,0.440330,0.385237,0.321376,-0.321566,-1.744855,-0.555411
4,5,business,147,-3.208512,0.380261,66.092705,-35.834000,0.324894,0.000385,6.352096,...,0.223714,0.178550,0.177725,0.234821,0.399142,0.343519,0.295413,-0.645753,-1.341985,-0.605482
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,7315,movie,158,1.374488,-1.621203,-28.808765,11.856544,0.158955,0.000110,10.326422,...,0.497821,0.500031,0.477741,0.481328,0.575820,0.563288,0.540482,-1.192416,1.312018,0.698750
7019,7316,movie,149,-0.589877,-0.506263,-42.102474,-8.204050,0.063389,0.000009,6.703302,...,0.388586,0.389387,0.358348,0.366467,0.488602,0.447671,0.428157,-0.999023,1.237880,0.928317
7020,7317,movie,110,0.057531,-0.210220,-14.665745,28.933609,0.161935,0.000023,6.471377,...,0.335177,0.335056,0.319898,0.334323,0.434197,0.410456,0.382911,-0.732479,1.445922,0.655112
7021,7318,movie,139,-0.113747,-0.901559,-19.301165,46.332840,0.296780,0.000262,5.628524,...,0.316369,0.313044,0.290388,0.306138,0.397719,0.377855,0.344800,-1.589979,1.253510,-0.387252


In [14]:
posts_info = pd.concat(([posts_info, dists_df, post_df.TotalTfIdf]), axis=1)

# posts_info.drop(["text"], axis=1, inplace=True)

posts_info

,post_id,text,topic,TextCluster,DistanceToCluster_0,DistanceToCluster_1,DistanceToCluster_2,DistanceToCluster_3,DistanceToCluster_4,DistanceToCluster_5,DistanceToCluster_6,DistanceToCluster_7,DistanceToCluster_8,DistanceToCluster_9,DistanceToCluster_10,DistanceToCluster_11,DistanceToCluster_12,DistanceToCluster_13,DistanceToCluster_14,TotalTfIdf
0,1,UK economy facing major risks\n\nThe UK manufa...,business,4,3.372981,3.449238,3.613279,2.455638,1.475142,3.410075,3.401131,3.605847,3.482929,1.889863,2.990790,3.417781,2.330397,3.443002,2.839993,8.748129
1,2,Aids and climate top Davos agenda\n\nClimate c...,business,9,3.327286,3.129955,3.375149,2.374099,2.187458,3.291418,3.440351,3.347250,3.258184,1.672582,2.841858,3.274209,2.251417,2.983633,2.548337,11.878472
2,3,Asian quake hits European shares\n\nShares in ...,business,9,3.263707,3.136165,3.334628,2.418176,2.277349,3.308573,3.549620,3.340957,3.403425,1.582005,3.051923,3.366638,3.114794,2.971580,2.866056,12.675530
3,4,India power shares jump on debut\n\nShares in ...,business,9,3.517888,3.770270,3.167191,2.842223,2.716161,3.736576,3.747341,3.791014,4.074710,2.356338,3.256052,3.790800,3.462207,3.721126,3.368291,6.622786
4,5,Lacroix label bought by US firm\n\nLuxury good...,business,9,3.031014,2.830061,3.065462,2.014575,2.055249,2.789516,2.850697,3.022428,3.245411,1.733670,2.643581,2.753824,3.005219,2.636937,2.116556,6.352096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,7315,"OK, I would not normally watch a Farrelly brot...",movie,5,3.133314,1.883501,2.063278,2.706920,3.132088,1.244245,1.729265,2.939593,3.400325,2.921895,2.820483,1.799387,3.392599,3.053521,2.304960,10.326422
7019,7316,I give this movie 2 stars purely because of it...,movie,5,2.930391,1.788261,1.889994,2.420229,2.926131,0.979706,1.182690,2.589023,3.389828,2.943105,2.508392,1.742759,3.240019,3.199836,2.218261,6.703302
7020,7317,I cant believe this film was allowed to be mad...,movie,5,2.833125,2.049317,2.291133,2.788901,3.211434,1.417765,1.973036,2.373447,3.468269,3.166540,2.539971,2.006334,3.446405,3.158383,2.429204,6.471377
7021,7318,The version I saw of this film was the Blockbu...,movie,11,3.430785,1.730188,1.742197,2.960285,3.268011,1.571262,1.184330,3.292385,3.414608,3.187736,3.109109,1.137585,3.477916,3.218224,2.272495,5.628524


In [15]:
### Очищаем память

model.cpu()

del model
del tokenizer

del dataset
del loader

del embeddings
del centered
del pca
del pca_decomp
del post_df

In [16]:
import gc

gc.collect()

116

In [17]:
# posts_info.to_sql("a_maslennikov_post_features_lesson_4_22", con=connection, if_exists='replace')

In [18]:
posts_info.drop(["text"], axis=1, inplace=True)
posts_info

,post_id,topic,TextCluster,DistanceToCluster_0,DistanceToCluster_1,DistanceToCluster_2,DistanceToCluster_3,DistanceToCluster_4,DistanceToCluster_5,DistanceToCluster_6,DistanceToCluster_7,DistanceToCluster_8,DistanceToCluster_9,DistanceToCluster_10,DistanceToCluster_11,DistanceToCluster_12,DistanceToCluster_13,DistanceToCluster_14,TotalTfIdf
0,1,business,4,3.372981,3.449238,3.613279,2.455638,1.475142,3.410075,3.401131,3.605847,3.482929,1.889863,2.990790,3.417781,2.330397,3.443002,2.839993,8.748129
1,2,business,9,3.327286,3.129955,3.375149,2.374099,2.187458,3.291418,3.440351,3.347250,3.258184,1.672582,2.841858,3.274209,2.251417,2.983633,2.548337,11.878472
2,3,business,9,3.263707,3.136165,3.334628,2.418176,2.277349,3.308573,3.549620,3.340957,3.403425,1.582005,3.051923,3.366638,3.114794,2.971580,2.866056,12.675530
3,4,business,9,3.517888,3.770270,3.167191,2.842223,2.716161,3.736576,3.747341,3.791014,4.074710,2.356338,3.256052,3.790800,3.462207,3.721126,3.368291,6.622786
4,5,business,9,3.031014,2.830061,3.065462,2.014575,2.055249,2.789516,2.850697,3.022428,3.245411,1.733670,2.643581,2.753824,3.005219,2.636937,2.116556,6.352096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,7315,movie,5,3.133314,1.883501,2.063278,2.706920,3.132088,1.244245,1.729265,2.939593,3.400325,2.921895,2.820483,1.799387,3.392599,3.053521,2.304960,10.326422
7019,7316,movie,5,2.930391,1.788261,1.889994,2.420229,2.926131,0.979706,1.182690,2.589023,3.389828,2.943105,2.508392,1.742759,3.240019,3.199836,2.218261,6.703302
7020,7317,movie,5,2.833125,2.049317,2.291133,2.788901,3.211434,1.417765,1.973036,2.373447,3.468269,3.166540,2.539971,2.006334,3.446405,3.158383,2.429204,6.471377
7021,7318,movie,11,3.430785,1.730188,1.742197,2.960285,3.268011,1.571262,1.184330,3.292385,3.414608,3.187736,3.109109,1.137585,3.477916,3.218224,2.272495,5.628524


## Теперь приступаем к обработке действий

In [19]:
### Заберем 10 млн, сразу очистим и оставим только view


feed_data = pd.read_sql(
    """
    SELECT
        cast(extract(hour from timestamp) as int) as hour,
        cast(extract(month from timestamp) as int) as month,
        post_id,
        gender,
        age,
        country,
        city,
        exp_group,
        os,
        source,
        target
    FROM public.feed_data JOIN public.user_data ON public.feed_data.user_id = public.user_data.user_id
    WHERE action = 'view'
    LIMIT 10000000
    """,
    con=connection
)

feed_data.head()

,hour,month,post_id,gender,age,country,city,exp_group,os,source,target
0,13,10,983,1,18,Russia,Domodedovo,2,iOS,ads,0
1,14,10,3475,1,18,Russia,Domodedovo,2,iOS,ads,0
2,14,10,3886,1,18,Russia,Domodedovo,2,iOS,ads,0
3,14,10,5379,1,18,Russia,Domodedovo,2,iOS,ads,0
4,14,10,1365,1,18,Russia,Domodedovo,2,iOS,ads,0


In [20]:
from catboost import CatBoostClassifier

object_cols = [
    'topic', 'TextCluster', 'gender', 'country',
    'city', 'exp_group', 'hour', 'month',
    'os', 'source'
]

catboost = CatBoostClassifier(
    iterations=200,
    learning_rate=1,
    depth=2,
    random_seed=12345612,
    thread_count=-1,
    task_type="GPU"
)

feed_data = pd.merge(
    feed_data,
    posts_info,
    on='post_id',
    how='left'
)

del posts_info

feed_data.drop(['post_id'], axis=1, inplace=True)

catboost.fit(X=feed_data.drop(['target'], axis=1), y=feed_data['target'], cat_features=object_cols)

catboost.save_model(
    'catboost_model',
    format="cbm"                  
)

0:	learn: 0.3630571	total: 283ms	remaining: 56.3s
1:	learn: 0.3550332	total: 539ms	remaining: 53.4s
2:	learn: 0.3535784	total: 774ms	remaining: 50.8s
3:	learn: 0.3529531	total: 1.03s	remaining: 50.5s
4:	learn: 0.3527287	total: 1.24s	remaining: 48.3s
5:	learn: 0.3516139	total: 1.47s	remaining: 47.7s
6:	learn: 0.3507214	total: 1.68s	remaining: 46.4s
7:	learn: 0.3506147	total: 1.9s	remaining: 45.5s
8:	learn: 0.3504213	total: 2.12s	remaining: 44.9s
9:	learn: 0.3503276	total: 2.34s	remaining: 44.5s
10:	learn: 0.3500488	total: 2.54s	remaining: 43.7s
11:	learn: 0.3499600	total: 2.75s	remaining: 43s
12:	learn: 0.3494120	total: 2.95s	remaining: 42.5s
13:	learn: 0.3492238	total: 3.16s	remaining: 42s
14:	learn: 0.3489585	total: 3.37s	remaining: 41.5s
15:	learn: 0.3487837	total: 3.61s	remaining: 41.5s
16:	learn: 0.3487505	total: 3.84s	remaining: 41.3s
17:	learn: 0.3481510	total: 4.04s	remaining: 40.9s
18:	learn: 0.3480733	total: 4.25s	remaining: 40.5s
19:	learn: 0.3480354	total: 4.48s	remaining: 4

161:	learn: 0.3451344	total: 34.7s	remaining: 8.14s
162:	learn: 0.3451297	total: 34.9s	remaining: 7.92s
163:	learn: 0.3451197	total: 35.1s	remaining: 7.7s
164:	learn: 0.3451141	total: 35.3s	remaining: 7.49s
165:	learn: 0.3451082	total: 35.5s	remaining: 7.27s
166:	learn: 0.3451029	total: 35.7s	remaining: 7.05s
167:	learn: 0.3450978	total: 35.9s	remaining: 6.84s
168:	learn: 0.3450414	total: 36.1s	remaining: 6.63s
169:	learn: 0.3449855	total: 36.3s	remaining: 6.41s
170:	learn: 0.3449079	total: 36.5s	remaining: 6.19s
171:	learn: 0.3448795	total: 36.7s	remaining: 5.97s
172:	learn: 0.3448720	total: 36.9s	remaining: 5.76s
173:	learn: 0.3448622	total: 37.1s	remaining: 5.54s
174:	learn: 0.3448592	total: 37.3s	remaining: 5.33s
175:	learn: 0.3448518	total: 37.5s	remaining: 5.12s
176:	learn: 0.3448493	total: 37.7s	remaining: 4.9s
177:	learn: 0.3448440	total: 37.9s	remaining: 4.68s
178:	learn: 0.3448379	total: 38.1s	remaining: 4.47s
179:	learn: 0.3448248	total: 38.3s	remaining: 4.26s
180:	learn: 0.

In [21]:
catboost.feature_names_

['hour',
 'month',
 'gender',
 'age',
 'country',
 'city',
 'exp_group',
 'os',
 'source',
 'topic',
 'TextCluster',
 'DistanceToCluster_0',
 'DistanceToCluster_1',
 'DistanceToCluster_2',
 'DistanceToCluster_3',
 'DistanceToCluster_4',
 'DistanceToCluster_5',
 'DistanceToCluster_6',
 'DistanceToCluster_7',
 'DistanceToCluster_8',
 'DistanceToCluster_9',
 'DistanceToCluster_10',
 'DistanceToCluster_11',
 'DistanceToCluster_12',
 'DistanceToCluster_13',
 'DistanceToCluster_14',
 'TotalTfIdf']

In [22]:
# model.save_model('test_catboost_model', format="cbm")